# * Adhoc : 2025 Sales Performance

In [2]:
import os
import glob
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2025, 6, 9)

## ETL Process

### History (since Jan 2025)

#### 202412 : ADHOC_RAW_SALE_PERFORMANCE_MTH
    as of December 2024
    
    Truncate -> Insert

In [ ]:
# Data Source

print(f'\nJob Start...')

query = """
    /*** Step 1 : Create Rawdata to "ADHOC_RAW_SALE_PERFORMANCE_MTH" ***/
    -----------------------------------------------------------------------------------------------------------------------

    WITH W_VAR AS
    (
        SELECT V_MONTH_START, V_MONTH_END
            , SUBSTR(V_MONTH_START,1,4)-1 || SUBSTR(V_MONTH_START,5,2) AS V_PREV_MONTH_START
            , SUBSTR(V_MONTH_END,1,4)-1 || SUBSTR(V_MONTH_END,5,2) AS V_PREV_MONTH_END
            , SUBSTR(V_MONTH_START,1,4)-1 AS V_PREV_YEAR
            , SUBSTR(V_MONTH_START,1,4) AS V_AS_YEAR
            
        FROM (SELECT 202410 AS V_MONTH_START, 202412 AS V_MONTH_END FROM DUAL) TMP
    )
    -----------------------------------------------------------------------------------------------------------------------

    , W_SALE_AGG AS 
    (
        -->> Rawdata 1 : Y2023
        SELECT TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, METRIC_CD, METRIC_NAME, PRODUCT_GRP, COMP_CD, AREA_TYPE, AREA_CD, AREA_NAME
            , SUM(ACTUAL_SNAP) AS ACTUAL_SNAP, SUM(TARGET_SNAP) AS TARGET_SNAP, MAX(PPN_TM) AS PPN_TM
        FROM GEOSPCAPPO.AGG_PERF_NEWCO
        WHERE METRIC_CD IN ('DB1R000900', 'DB2R000500', 'TB1R000900', 'TB2R000500', 'TB3R000600', 'TB4R001000')
        AND AREA_TYPE IN ('P', 'G', 'H')
        AND TM_KEY_MTH BETWEEN (SELECT V_PREV_MONTH_START FROM W_VAR) AND (SELECT V_PREV_MONTH_END FROM W_VAR)
        GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, METRIC_CD, METRIC_NAME, PRODUCT_GRP, COMP_CD, AREA_TYPE, AREA_CD, AREA_NAME
        
        UNION ALL 
        
        -->> Rawdata 2 : Y2024 (P)
        SELECT TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, METRIC_CD, METRIC_NAME, PRODUCT_GRP, COMP_CD--, AREA_TYPE, AREA_CD, AREA_NAME
            , 'P' AS AREA_TYPE, 'P' AS AREA_CD, 'Nationwide' AS AREA_NAME
            , SUM(ACTUAL_SNAP) AS ACTUAL_SNAP, SUM(TARGET_SNAP) AS TARGET_SNAP, MAX(PPN_TM) AS PPN_TM
        FROM GEOSPCAPPO.AGG_PERF_NEWCO
        WHERE METRIC_CD IN ('DB1R000900', 'DB2R000500', 'TB1R000900', 'TB2R000500', 'TB3R000600', 'TB4R001000')
        AND AREA_TYPE = 'C'
        AND TM_KEY_MTH BETWEEN (SELECT V_MONTH_START FROM W_VAR) AND (SELECT V_MONTH_END FROM W_VAR)
        GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, METRIC_CD, METRIC_NAME, PRODUCT_GRP, COMP_CD, AREA_TYPE, AREA_CD, AREA_NAME
    ) -->> W_SALE_AGG
    -----------------------------------------------------------------------------------------------------------------------

    , W_SALE_FCT AS 
    (
        -->> Rawdata 3 : Y2024 (G, H from source)
        SELECT TMP_TIME.TM_KEY_YR, TMP_TIME.TM_KEY_QTR, CAST(TMP_ACT.TM_KEY_MTH AS INT) AS TM_KEY_MTH
            , TMP_ACT.METRIC_CD
            , TMP_METRIC.METRIC_NAME, TMP_METRIC.PRODUCT_GRP
            , TMP_ACT.COMP_CD, TMP_ACT.AREA_TYPE, TMP_ACT.AREA_CD--, TMP_ACT.AREA_DESC AS AREA_NAME
            , NULL AS AREA_NAME
            , TMP_ACT.ACTUAL AS ACTUAL_SNAP
            , TMP_TGT.TARGET AS TARGET_SNAP
            , TMP_ACT.LOAD_DATE AS PPN_TM
            
        FROM (
            --DTAC : Actual
            SELECT SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH, METRIC_CD, METRIC_NAME, COMP_CD, AREA_TYPE, AREA_CD
                , SUM(ACTUAL) ACTUAL, MAX(LOAD_DATE) LOAD_DATE
            FROM GEOSPCAPPO.FCT_KPI_NEWCO_DIAMOND_ACTUAL_SOURCE
            WHERE METRIC_CD IN ('DB1R000900', 'DB2R000500')
            AND SUBSTR(TM_KEY_DAY,1,6) BETWEEN (SELECT V_MONTH_START FROM W_VAR) AND (SELECT V_MONTH_END FROM W_VAR)
            AND AREA_TYPE IN ('G', 'H')
            AND NOT (AREA_CD IS NULL OR AREA_CD IN ('-1', '-99', 'Z00', '9GZ', 'GX0', 'Unidentified', 'True Corp'))
            GROUP BY SUBSTR(TM_KEY_DAY,1,6), METRIC_CD, METRIC_NAME, COMP_CD, AREA_TYPE, AREA_CD
            
            UNION ALL 
            --TRUE : Actual
            SELECT SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH, METRIC_CD, METRIC_NAME, COMP_CD, AREA_TYPE, AREA_CD
                , SUM(ACTUAL) ACTUAL, MAX(LOAD_DATE) LOAD_DATE
            FROM GEOSPCAPPO.FCT_KPI_NEWCO_CDS_ACTUAL_SOURCE
            WHERE METRIC_CD IN ('TB1R000900', 'TB2R000500', 'TB3R000600', 'TB4R001000')
            AND SUBSTR(TM_KEY_DAY,1,6) BETWEEN (SELECT V_MONTH_START FROM W_VAR) AND (SELECT V_MONTH_END FROM W_VAR)
            AND AREA_TYPE IN ('G', 'H')
            AND NOT (AREA_CD IS NULL OR AREA_CD IN ('-1', '-99', 'Z00', '9GZ', 'GX0', 'Unidentified', 'True Corp'))
            GROUP BY SUBSTR(TM_KEY_DAY,1,6), METRIC_CD, METRIC_NAME, COMP_CD, AREA_TYPE, AREA_CD
        ) TMP_ACT
        
        LEFT JOIN (
            --DTAC : Target
            SELECT SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH, METRIC_CD, AREA_CD, SUM(TARGET) TARGET
            FROM GEOSPCAPPO.FCT_KPI_NEWCO_DIAMOND_TARGET_SOURCE
            WHERE METRIC_CD IN ('DB1R000900', 'DB2R000500')
            AND SUBSTR(TM_KEY_DAY,1,6) BETWEEN (SELECT V_MONTH_START FROM W_VAR) AND (SELECT V_MONTH_END FROM W_VAR)
            AND AREA_TYPE IN ('G', 'H')
            AND NOT (AREA_CD IS NULL OR AREA_CD IN ('-1', '-99', 'Z00', '9GZ', 'GX0', 'Unidentified', 'True Corp'))
            GROUP BY SUBSTR(TM_KEY_DAY,1,6), METRIC_CD, AREA_CD
            
            UNION ALL 
            --TRUE : Target
            SELECT SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH, METRIC_CD, AREA_CD, SUM(TARGET) TARGET
            FROM GEOSPCAPPO.FCT_KPI_NEWCO_CDS_TARGET_SOURCE
            WHERE METRIC_CD IN ('TB1R000900', 'TB2R000500', 'TB3R000600', 'TB4R001000')
            AND SUBSTR(TM_KEY_DAY,1,6) BETWEEN (SELECT V_MONTH_START FROM W_VAR) AND (SELECT V_MONTH_END FROM W_VAR)
            AND AREA_TYPE IN ('G', 'H')
            AND NOT (AREA_CD IS NULL OR AREA_CD IN ('-1', '-99', 'Z00', '9GZ', 'GX0', 'Unidentified', 'True Corp'))
            GROUP BY SUBSTR(TM_KEY_DAY,1,6), METRIC_CD, AREA_CD
        ) TMP_TGT
            ON TMP_ACT.TM_KEY_MTH = TMP_TGT.TM_KEY_MTH
            AND TMP_ACT.METRIC_CD = TMP_TGT.METRIC_CD
            AND TMP_ACT.AREA_CD = TMP_TGT.AREA_CD
            
        LEFT JOIN (
            SELECT DISTINCT METRIC_CD, METRIC_NAME, PRODUCT_GRP 
            FROM GEOSPCAPPO.AGG_PERF_NEWCO
            WHERE TM_KEY_MTH = (SELECT V_MONTH_END FROM W_VAR)
            AND METRIC_CD IN ('DB1R000900', 'DB2R000500', 'TB1R000900', 'TB2R000500', 'TB3R000600', 'TB4R001000')
        ) TMP_METRIC
            ON TMP_ACT.METRIC_CD = TMP_METRIC.METRIC_CD
            
        LEFT JOIN (
            SELECT DISTINCT TM_KEY_MTH, TM_KEY_QTR, TM_KEY_YR 
            FROM CDSAPPO.DIM_TIME
            WHERE TM_KEY_MTH BETWEEN (SELECT V_PREV_MONTH_START FROM W_VAR) AND (SELECT V_MONTH_END FROM W_VAR)
        ) TMP_TIME
            ON TMP_ACT.TM_KEY_MTH = TMP_TIME.TM_KEY_MTH
    ) -->> W_SALE_FCT
    -----------------------------------------------------------------------------------------------------------------------

    -->> Output

    SELECT /*+ PARALLEL(4)*/ 
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
        , CASE WHEN AREA_TYPE = 'P' THEN 1 WHEN AREA_TYPE = 'G' THEN 2 WHEN AREA_TYPE = 'H' THEN 3 ELSE 0 END AREA_NO
        , AREA_TYPE, SALE_TXN.AREA_CD, TMP_AREA.AREA_NAME
        , ACTUAL_SNAP, TARGET_SNAP, PPN_TM

    FROM (
        SELECT *  FROM W_SALE_AGG
        UNION ALL 
        SELECT * FROM W_SALE_FCT
    ) SALE_TXN

    LEFT JOIN (
        SELECT DISTINCT AREA_CD, AREA_NAME  
        FROM GEOSPCAPPO.AGG_PERF_NEWCO
        WHERE TM_KEY_MTH = (SELECT V_MONTH_END FROM W_VAR)
    ) TMP_AREA
        ON SALE_TXN.AREA_CD = TMP_AREA.AREA_CD

    --ORDER BY TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, AREA_NO, AREA_CD
"""

try:
    # TDMDBPR
    dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
    conn = oracledb.connect(dsn)
    print(f'\n{TDMDBPR_db} : Connected')
    cur = conn.cursor()

    # Create Rawdata
    print(f'\n Processing...')
    cur.execute(query)
    df = pd.DataFrame(cur.fetchall(), columns=['TM_KEY_YR', 'TM_KEY_QTR', 'TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP', 'PPN_TM'])
    cur.close()
    df = df.replace(np.nan, None)
    df = df.astype(dtype={'TM_KEY_YR':'float', 'TM_KEY_QTR':'float', 'TM_KEY_MTH':'float', 'AREA_NO':'float'})
    rows = [tuple(x) for x in df.values]
    print(f'\n -> CREATE : Rawdata : Done !')
    
    # AKPIPRD
    op_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
    op_conn = oracledb.connect(op_dsn)
    print(f'\n{AKPIPRD_db} : Connected')
    op_cur = op_conn.cursor()

    # Truncate
    op_cur.execute("TRUNCATE TABLE AUTOKPI.ADHOC_RAW_SALE_PERFORMANCE_MTH")
    print(f'\n -> TRUNCATE : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !')

    # Insert
    op_cur.executemany("INSERT INTO ADHOC_RAW_SALE_PERFORMANCE_MTH\
                    (TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP, PPN_TM)\
                    VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14)", rows)
    op_cur.close()
    op_conn.commit()
    print(f'\n -> INSERT : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !\n\n{df.shape[0]} rows, {df.shape[1]} columns')


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')

    op_conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    
    print(f'\nJob Done !!!')

#### 202501 : ADHOC_RAW_SALE_PERFORMANCE_MTH
    as of January 2025
    
    Truncate -> Insert

In [ ]:
# Data Source

print(f'\nJob Start...')

query = """
    /*** Step 1 : Create Rawdata to "ADHOC_RAW_SALE_PERFORMANCE_MTH" ***/
    -----------------------------------------------------------------------------------------------------------------------

    WITH W_VAR AS
    (
        SELECT V_MONTH_START, V_MONTH_END
            , SUBSTR(V_MONTH_START,1,4)-1 || SUBSTR(V_MONTH_START,5,2) AS V_PREV_MONTH_START
            , SUBSTR(V_MONTH_END,1,4)-1 || SUBSTR(V_MONTH_END,5,2) AS V_PREV_MONTH_END
            , SUBSTR(V_MONTH_START,1,4)-1 AS V_PREV_YEAR
            , SUBSTR(V_MONTH_END,1,4) AS V_AS_YEAR
            
        FROM (SELECT 202412 AS V_MONTH_START, 202501 AS V_MONTH_END FROM DUAL) TMP
    )
    -----------------------------------------------------------------------------------------------------------------------

    , W_SALE_AGG AS 
    (
        SELECT TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
            , CASE WHEN AREA_TYPE = 'P' THEN 1 WHEN AREA_TYPE = 'G' THEN 2 WHEN AREA_TYPE = 'H' THEN 3 ELSE 0 END AREA_NO
            , AREA_TYPE, AREA_CD, AREA_NAME
            , SUM(ACTUAL_SNAP) AS ACTUAL_SNAP, SUM(TARGET_SNAP) AS TARGET_SNAP, MAX(PPN_TM) AS PPN_TM
        FROM GEOSPCAPPO.AGG_PERF_NEWCO
        WHERE METRIC_CD IN (
            'DB1R000900CS' --Prepaid Inflow M1 : DTAC
            , 'TB1R000900CS' --Prepaid Inflow M1 : TMH
            , 'DB2R010500CS' --Postpaid Inflow M1 B2C : DTAC
            , 'DB2R020500CS' --Postpaid Inflow M1 B2B : DTAC
            , 'TB2R010500CS' --Postpaid Inflow M1 B2C : TMH
            , 'TB2R020500CS' --Postpaid Inflow M1 B2B : TMH
            , 'TB3R000601CS' --TOL Inflow M1 Connected : Consumer
            , 'TB3R000602CS' --TOL Inflow M1 Connected : DataService
            )
        AND AREA_TYPE IN ('P', 'G', 'H')
        AND ( TM_KEY_MTH BETWEEN (SELECT V_PREV_MONTH_START FROM W_VAR) AND (SELECT V_PREV_MONTH_END FROM W_VAR) 
            OR TM_KEY_MTH BETWEEN (SELECT V_MONTH_START FROM W_VAR) AND (SELECT V_MONTH_END FROM W_VAR) )
        GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_NAME
    ) -->> W_SALE_AGG
    -----------------------------------------------------------------------------------------------------------------------

    -->> Output

    SELECT /*+PARALLEL(8)*/ 
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP, PPN_TM
    FROM W_SALE_AGG

    --ORDER BY TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, AREA_NO, AREA_CD
"""

try:
    # TDMDBPR
    dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
    conn = oracledb.connect(dsn)
    print(f'\n{TDMDBPR_db} : Connected')
    cur = conn.cursor()

    # Create Rawdata
    print(f'\n Processing...')
    cur.execute(query)
    df = pd.DataFrame(cur.fetchall(), columns=['TM_KEY_YR', 'TM_KEY_QTR', 'TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP', 'PPN_TM'])
    cur.close()
    df = df.replace(np.nan, None)
    df = df.astype(dtype={'TM_KEY_YR':'float', 'TM_KEY_QTR':'float', 'TM_KEY_MTH':'float', 'AREA_NO':'float'})
    rows = [tuple(x) for x in df.values]
    print(f'\n -> CREATE : Rawdata : Done !')
    
    # AKPIPRD
    op_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
    op_conn = oracledb.connect(op_dsn)
    print(f'\n{AKPIPRD_db} : Connected')
    op_cur = op_conn.cursor()

    # Truncate
    op_cur.execute("TRUNCATE TABLE AUTOKPI.ADHOC_RAW_SALE_PERFORMANCE_MTH")
    print(f'\n -> TRUNCATE : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !')

    # Insert
    op_cur.executemany("INSERT INTO ADHOC_RAW_SALE_PERFORMANCE_MTH\
                    (TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP, PPN_TM)\
                    VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14)", rows)
    op_cur.close()
    op_conn.commit()
    print(f'\n -> INSERT : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !\n\n{df.shape[0]} rows, {df.shape[1]} columns')


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')

    op_conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    
    print(f'\nJob Done !!!')

#### 202502 : ADHOC_RAW_SALE_PERFORMANCE_MTH
    as of February 2025
    
    Truncate -> Insert

In [ ]:
# Data Source

print(f'\nJob Start...')

query = """
    /*** Step 1 : Create Rawdata to "ADHOC_RAW_SALE_PERFORMANCE_MTH" ***/
    -----------------------------------------------------------------------------------------------------------------------

    WITH W_VAR AS
    (
        SELECT V_MONTH_START, V_MONTH_END
            , SUBSTR(V_MONTH_START,1,4)-1 || SUBSTR(V_MONTH_START,5,2) AS V_PREV_MONTH_START
            , SUBSTR(V_MONTH_END,1,4)-1 || SUBSTR(V_MONTH_END,5,2) AS V_PREV_MONTH_END
            , SUBSTR(V_MONTH_START,1,4)-1 AS V_PREV_YEAR
            , SUBSTR(V_MONTH_END,1,4) AS V_AS_YEAR
            
        FROM (SELECT 202501 AS V_MONTH_START, 202502 AS V_MONTH_END FROM DUAL) TMP
    )
    -----------------------------------------------------------------------------------------------------------------------

    , W_SALE_AGG AS 
    (
        SELECT TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
            , CASE WHEN AREA_TYPE = 'P' THEN 1 WHEN AREA_TYPE = 'G' THEN 2 WHEN AREA_TYPE = 'H' THEN 3 ELSE 0 END AREA_NO
            , AREA_TYPE, AREA_CD, AREA_NAME
            , SUM(ACTUAL_SNAP) AS ACTUAL_SNAP, SUM(TARGET_SNAP) AS TARGET_SNAP, MAX(PPN_TM) AS PPN_TM
        FROM GEOSPCAPPO.AGG_PERF_NEWCO
        WHERE METRIC_CD IN (
            'DB1R000900CS' --Prepaid Inflow M1 : DTAC
            , 'TB1R000900CS' --Prepaid Inflow M1 : TMH
            , 'DB2R010500CS' --Postpaid Inflow M1 B2C : DTAC
            , 'DB2R020500CS' --Postpaid Inflow M1 B2B : DTAC
            , 'TB2R010500CS' --Postpaid Inflow M1 B2C : TMH
            , 'TB2R020500CS' --Postpaid Inflow M1 B2B : TMH
            , 'TB3R000601CS' --TOL Inflow M1 Connected : Consumer
            , 'TB3R000602CS' --TOL Inflow M1 Connected : DataService
            )
        AND AREA_TYPE IN ('P', 'G', 'H')
        AND ( TM_KEY_MTH BETWEEN (SELECT V_PREV_MONTH_START FROM W_VAR) AND (SELECT V_PREV_MONTH_END FROM W_VAR) 
            OR TM_KEY_MTH BETWEEN (SELECT V_MONTH_START FROM W_VAR) AND (SELECT V_MONTH_END FROM W_VAR) )
        GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_NAME
    ) -->> W_SALE_AGG
    -----------------------------------------------------------------------------------------------------------------------

    -->> Output

    SELECT /*+PARALLEL(8)*/ 
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP, PPN_TM
    FROM W_SALE_AGG

    --ORDER BY TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, AREA_NO, AREA_CD
"""

try:
    # TDMDBPR
    dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
    conn = oracledb.connect(dsn)
    print(f'\n{TDMDBPR_db} : Connected')
    cur = conn.cursor()

    # Create Rawdata
    print(f'\n Processing...')
    cur.execute(query)
    df = pd.DataFrame(cur.fetchall(), columns=['TM_KEY_YR', 'TM_KEY_QTR', 'TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP', 'PPN_TM'])
    cur.close()
    df = df.replace(np.nan, None)
    df = df.astype(dtype={'TM_KEY_YR':'float', 'TM_KEY_QTR':'float', 'TM_KEY_MTH':'float', 'AREA_NO':'float'})
    rows = [tuple(x) for x in df.values]
    print(f'\n -> CREATE : Rawdata : Done !')
    
    # AKPIPRD
    op_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
    op_conn = oracledb.connect(op_dsn)
    print(f'\n{AKPIPRD_db} : Connected')
    op_cur = op_conn.cursor()

    # Truncate
    op_cur.execute("TRUNCATE TABLE AUTOKPI.ADHOC_RAW_SALE_PERFORMANCE_MTH")
    print(f'\n -> TRUNCATE : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !')

    # Insert
    op_cur.executemany("INSERT INTO ADHOC_RAW_SALE_PERFORMANCE_MTH\
                    (TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP, PPN_TM)\
                    VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14)", rows)
    op_cur.close()
    op_conn.commit()
    print(f'\n -> INSERT : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !\n\n{df.shape[0]} rows, {df.shape[1]} columns')


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')

    op_conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    
    print(f'\nJob Done !!!')

#### 202503 : ADHOC_RAW_SALE_PERFORMANCE_MTH
    as of March 2025
    
    Truncate -> Insert

In [ ]:
# Data Source

print(f'\nJob Start...')

query = """
    /*** Step 1 : Create Rawdata to "ADHOC_RAW_SALE_PERFORMANCE_MTH" ***/
    -----------------------------------------------------------------------------------------------------------------------

    WITH W_VAR AS
    (
        SELECT V_MONTH_START, V_MONTH_END
            , SUBSTR(V_MONTH_START,1,4)-1 || SUBSTR(V_MONTH_START,5,2) AS V_PREV_MONTH_START
            , SUBSTR(V_MONTH_END,1,4)-1 || SUBSTR(V_MONTH_END,5,2) AS V_PREV_MONTH_END
            , SUBSTR(V_MONTH_START,1,4)-1 AS V_PREV_YEAR
            , SUBSTR(V_MONTH_END,1,4) AS V_AS_YEAR
            
        FROM (SELECT 202502 AS V_MONTH_START, 202503 AS V_MONTH_END FROM DUAL) TMP
    )
    -----------------------------------------------------------------------------------------------------------------------

    , W_SALE_AGG AS 
    (
        SELECT TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
            , CASE WHEN AREA_TYPE = 'P' THEN 1 WHEN AREA_TYPE = 'G' THEN 2 WHEN AREA_TYPE = 'H' THEN 3 ELSE 0 END AREA_NO
            , AREA_TYPE, AREA_CD, AREA_NAME
            , SUM(ACTUAL_SNAP) AS ACTUAL_SNAP, SUM(TARGET_SNAP) AS TARGET_SNAP, MAX(PPN_TM) AS PPN_TM
        FROM GEOSPCAPPO.AGG_PERF_NEWCO
        WHERE METRIC_CD IN (
            'DB1R000900CS' --Prepaid Inflow M1 : DTAC
            , 'TB1R000900CS' --Prepaid Inflow M1 : TMH
            , 'DB2R010500CS' --Postpaid Inflow M1 B2C : DTAC
            , 'DB2R020500CS' --Postpaid Inflow M1 B2B : DTAC
            , 'TB2R010500CS' --Postpaid Inflow M1 B2C : TMH
            , 'TB2R020500CS' --Postpaid Inflow M1 B2B : TMH
            , 'TB3R000601CS' --TOL Inflow M1 Connected : Consumer
            , 'TB3R000602CS' --TOL Inflow M1 Connected : DataService
            )
        AND AREA_TYPE IN ('P', 'G', 'H')
        AND ( TM_KEY_MTH BETWEEN (SELECT V_PREV_MONTH_START FROM W_VAR) AND (SELECT V_PREV_MONTH_END FROM W_VAR) 
            OR TM_KEY_MTH BETWEEN (SELECT V_MONTH_START FROM W_VAR) AND (SELECT V_MONTH_END FROM W_VAR) )
        GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_NAME
    ) -->> W_SALE_AGG
    -----------------------------------------------------------------------------------------------------------------------

    -->> Output

    SELECT /*+PARALLEL(8)*/ 
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP, PPN_TM
    FROM W_SALE_AGG

    --ORDER BY TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, AREA_NO, AREA_CD
"""

try:
    # TDMDBPR
    dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
    conn = oracledb.connect(dsn)
    print(f'\n{TDMDBPR_db} : Connected')
    cur = conn.cursor()

    # Create Rawdata
    print(f'\n Processing...')
    cur.execute(query)
    df = pd.DataFrame(cur.fetchall(), columns=['TM_KEY_YR', 'TM_KEY_QTR', 'TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP', 'PPN_TM'])
    cur.close()
    df = df.replace(np.nan, None)
    df = df.astype(dtype={'TM_KEY_YR':'float', 'TM_KEY_QTR':'float', 'TM_KEY_MTH':'float', 'AREA_NO':'float'})
    rows = [tuple(x) for x in df.values]
    print(f'\n -> CREATE : Rawdata : Done !')
    
    # AKPIPRD
    op_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
    op_conn = oracledb.connect(op_dsn)
    print(f'\n{AKPIPRD_db} : Connected')
    op_cur = op_conn.cursor()

    # Truncate
    op_cur.execute("TRUNCATE TABLE AUTOKPI.ADHOC_RAW_SALE_PERFORMANCE_MTH")
    print(f'\n -> TRUNCATE : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !')

    # Insert
    op_cur.executemany("INSERT INTO ADHOC_RAW_SALE_PERFORMANCE_MTH\
                    (TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP, PPN_TM)\
                    VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14)", rows)
    op_cur.close()
    op_conn.commit()
    print(f'\n -> INSERT : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !\n\n{df.shape[0]} rows, {df.shape[1]} columns')


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')

    op_conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    
    print(f'\nJob Done !!!')


Job Start...

TDMDBPR : Connected

 Processing...

 -> CREATE : Rawdata : Done !

AKPIPRD : Connected

 -> TRUNCATE : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !

 -> INSERT : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !

2400 rows, 14 columns

TDMDBPR : Disconnected

AKPIPRD : Disconnected

Job Done !!!


#### 202504 : ADHOC_RAW_SALE_PERFORMANCE_MTH
    as of April 2025
    
    Truncate -> Insert

In [ ]:
# Data Source

print(f'\nJob Start...')

query = """
    /*** Step 1 : Create Rawdata to "ADHOC_RAW_SALE_PERFORMANCE_MTH" ***/
    -----------------------------------------------------------------------------------------------------------------------

    WITH W_VAR AS
    (
        SELECT V_MONTH_START, V_MONTH_END
            , SUBSTR(V_MONTH_START,1,4)-1 || SUBSTR(V_MONTH_START,5,2) AS V_PREV_MONTH_START
            , SUBSTR(V_MONTH_END,1,4)-1 || SUBSTR(V_MONTH_END,5,2) AS V_PREV_MONTH_END
            , SUBSTR(V_MONTH_START,1,4)-1 AS V_PREV_YEAR
            , SUBSTR(V_MONTH_END,1,4) AS V_AS_YEAR
            
        FROM (SELECT 202502 AS V_MONTH_START, 202504 AS V_MONTH_END FROM DUAL) TMP
    )
    -----------------------------------------------------------------------------------------------------------------------

    , W_SALE_AGG AS 
    (
        SELECT TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
            , CASE WHEN AREA_TYPE = 'P' THEN 1 WHEN AREA_TYPE = 'G' THEN 2 WHEN AREA_TYPE = 'H' THEN 3 ELSE 0 END AREA_NO
            , AREA_TYPE, AREA_CD, AREA_NAME
            , SUM(ACTUAL_SNAP) AS ACTUAL_SNAP, SUM(TARGET_SNAP) AS TARGET_SNAP, MAX(PPN_TM) AS PPN_TM
        FROM GEOSPCAPPO.AGG_PERF_NEWCO
        WHERE METRIC_CD IN (
            'DB1R000900CS' --Prepaid Inflow M1 : DTAC
            , 'TB1R000900CS' --Prepaid Inflow M1 : TMH
            , 'DB2R010500CS' --Postpaid Inflow M1 B2C : DTAC
            , 'DB2R020500CS' --Postpaid Inflow M1 B2B : DTAC
            , 'TB2R010500CS' --Postpaid Inflow M1 B2C : TMH
            , 'TB2R020500CS' --Postpaid Inflow M1 B2B : TMH
            , 'TB3R000601CS' --TOL Inflow M1 Connected : Consumer
            , 'TB3R000602CS' --TOL Inflow M1 Connected : DataService
            )
        AND AREA_TYPE IN ('P', 'G', 'H')
        AND ( TM_KEY_MTH BETWEEN (SELECT V_PREV_MONTH_START FROM W_VAR) AND (SELECT V_PREV_MONTH_END FROM W_VAR) 
            OR TM_KEY_MTH BETWEEN (SELECT V_MONTH_START FROM W_VAR) AND (SELECT V_MONTH_END FROM W_VAR) )
        GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_NAME
    ) -->> W_SALE_AGG
    -----------------------------------------------------------------------------------------------------------------------

    -->> Output

    SELECT /*+PARALLEL(8)*/ 
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP, PPN_TM
    FROM W_SALE_AGG

    --ORDER BY TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, AREA_NO, AREA_CD
"""

try:
    # TDMDBPR
    dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
    conn = oracledb.connect(dsn)
    print(f'\n{TDMDBPR_db} : Connected')
    cur = conn.cursor()

    # Create Rawdata
    print(f'\n Processing...')
    cur.execute(query)
    df = pd.DataFrame(cur.fetchall(), columns=['TM_KEY_YR', 'TM_KEY_QTR', 'TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP', 'PPN_TM'])
    cur.close()
    df = df.replace(np.nan, None)
    df = df.astype(dtype={'TM_KEY_YR':'float', 'TM_KEY_QTR':'float', 'TM_KEY_MTH':'float', 'AREA_NO':'float'})
    rows = [tuple(x) for x in df.values]
    print(f'\n -> CREATE : Rawdata : Done !')
    
    # AKPIPRD
    op_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
    op_conn = oracledb.connect(op_dsn)
    print(f'\n{AKPIPRD_db} : Connected')
    op_cur = op_conn.cursor()

    # Truncate
    op_cur.execute("TRUNCATE TABLE AUTOKPI.ADHOC_RAW_SALE_PERFORMANCE_MTH")
    print(f'\n -> TRUNCATE : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !')

    # Insert
    op_cur.executemany("INSERT INTO ADHOC_RAW_SALE_PERFORMANCE_MTH\
                    (TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP, PPN_TM)\
                    VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14)", rows)
    op_cur.close()
    op_conn.commit()
    print(f'\n -> INSERT : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !\n\n{df.shape[0]} rows, {df.shape[1]} columns')


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')

    op_conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    
    print(f'\nJob Done !!!')


Job Start...

TDMDBPR : Connected

 Processing...

 -> CREATE : Rawdata : Done !

AKPIPRD : Connected

 -> TRUNCATE : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !

 -> INSERT : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !

3504 rows, 14 columns

TDMDBPR : Disconnected

AKPIPRD : Disconnected

Job Done !!!


### Step 1 : ADHOC_RAW_SALE_PERFORMANCE_MTH
    as of May 2025
    
    Truncate -> Insert

In [3]:
''' Execute transaction '''


print(f'\nJob Start...')

query = """
    /*** Step 1 : Create Rawdata to "ADHOC_RAW_SALE_PERFORMANCE_MTH" ***/
    -----------------------------------------------------------------------------------------------------------------------

    WITH W_VAR AS
    (
        SELECT V_MONTH_START, V_MONTH_END
            , SUBSTR(V_MONTH_START,1,4)-1 || SUBSTR(V_MONTH_START,5,2) AS V_PREV_MONTH_START
            , SUBSTR(V_MONTH_END,1,4)-1 || SUBSTR(V_MONTH_END,5,2) AS V_PREV_MONTH_END
            , SUBSTR(V_MONTH_START,1,4)-1 AS V_PREV_YEAR
            , SUBSTR(V_MONTH_END,1,4) AS V_AS_YEAR
            
        FROM (SELECT 202503 AS V_MONTH_START, 202505 AS V_MONTH_END FROM DUAL) TMP
    )
    -----------------------------------------------------------------------------------------------------------------------

    , W_SALE_AGG AS 
    (
        SELECT TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
            , CASE WHEN AREA_TYPE = 'P' THEN 1 WHEN AREA_TYPE = 'G' THEN 2 WHEN AREA_TYPE = 'H' THEN 3 ELSE 0 END AREA_NO
            , AREA_TYPE, AREA_CD, AREA_NAME
            , SUM(ACTUAL_SNAP) AS ACTUAL_SNAP, SUM(TARGET_SNAP) AS TARGET_SNAP, MAX(PPN_TM) AS PPN_TM
        FROM GEOSPCAPPO.AGG_PERF_NEWCO
        WHERE METRIC_CD IN (
            'DB1R000900CS' --Prepaid Inflow M1 : DTAC
            , 'TB1R000900CS' --Prepaid Inflow M1 : TMH
            , 'DB2R010500CS' --Postpaid Inflow M1 B2C : DTAC
            , 'DB2R020500CS' --Postpaid Inflow M1 B2B : DTAC
            , 'TB2R010500CS' --Postpaid Inflow M1 B2C : TMH
            , 'TB2R020500CS' --Postpaid Inflow M1 B2B : TMH
            , 'TB3R000601CS' --TOL Inflow M1 Connected : Consumer
            , 'TB3R000602CS' --TOL Inflow M1 Connected : DataService
            )
        AND AREA_TYPE IN ('P', 'G', 'H')
        AND ( TM_KEY_MTH BETWEEN (SELECT V_PREV_MONTH_START FROM W_VAR) AND (SELECT V_PREV_MONTH_END FROM W_VAR) 
            OR TM_KEY_MTH BETWEEN (SELECT V_MONTH_START FROM W_VAR) AND (SELECT V_MONTH_END FROM W_VAR) )
        GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_NAME
    ) -->> W_SALE_AGG
    -----------------------------------------------------------------------------------------------------------------------

    -->> Output

    SELECT /*+PARALLEL(8)*/ 
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP, PPN_TM
    FROM W_SALE_AGG

    --ORDER BY TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, AREA_NO, AREA_CD
"""

try:
    # TDMDBPR
    dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
    conn = oracledb.connect(dsn)
    print(f'\n{TDMDBPR_db} : Connected')
    cur = conn.cursor()

    # Create Rawdata
    print(f'\n Processing...')
    cur.execute(query)
    df = pd.DataFrame(cur.fetchall(), columns=['TM_KEY_YR', 'TM_KEY_QTR', 'TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP', 'PPN_TM'])
    cur.close()
    df = df.replace(np.nan, None)
    df = df.astype(dtype={'TM_KEY_YR':'float', 'TM_KEY_QTR':'float', 'TM_KEY_MTH':'float', 'AREA_NO':'float'})
    rows = [tuple(x) for x in df.values]
    print(f'\n -> CREATE : Rawdata : Done !')
    
    # AKPIPRD
    op_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
    op_conn = oracledb.connect(op_dsn)
    print(f'\n{AKPIPRD_db} : Connected')
    op_cur = op_conn.cursor()

    # Truncate
    op_cur.execute("TRUNCATE TABLE AUTOKPI.ADHOC_RAW_SALE_PERFORMANCE_MTH")
    print(f'\n -> TRUNCATE : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !')

    # Insert
    op_cur.executemany("INSERT INTO ADHOC_RAW_SALE_PERFORMANCE_MTH\
                    (TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP, PPN_TM)\
                    VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14)", rows)
    op_cur.close()
    op_conn.commit()
    print(f'\n -> INSERT : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !\n\n{df.shape[0]} rows, {df.shape[1]} columns')


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')

    op_conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    
    print(f'\nJob Done !!!')


Job Start...

TDMDBPR : Connected

 Processing...

 -> CREATE : Rawdata : Done !

AKPIPRD : Connected

 -> TRUNCATE : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !

 -> INSERT : "ADHOC_RAW_SALE_PERFORMANCE_MTH" : Done !

3504 rows, 14 columns

TDMDBPR : Disconnected

AKPIPRD : Disconnected

Job Done !!!


### Step 2 : Generate "Sales Performance Report"
    DB -> Excel file

In [4]:
# Output to Excel

print(f'\nJob Start...')

query = """
    /*** Sales Performance Report ***/

    WITH W_SALE_ACTUAL AS 
    (
        SELECT TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD--, METRIC_CD, METRIC_NAME
            , CASE 	WHEN METRIC_CD IN ('DB2R010500CS', 'DB2R020500CS') THEN 'DB2R000500CS' --Postpaid Inflow M1 : DTAC (Mockup B2C+B2B)
                    WHEN METRIC_CD IN ('TB2R010500CS', 'TB2R020500CS') THEN 'TB2R000500CS' --Postpaid Inflow M1 : TMH (Mockup B2C+B2B)
                    ELSE METRIC_CD END METRIC_CD
            , CASE 	WHEN METRIC_CD IN ('DB2R010500CS', 'DB2R020500CS') THEN 'Postpaid Inflow M1 : DTAC' --Mockup B2C+B2B
                    WHEN METRIC_CD IN ('TB2R010500CS', 'TB2R020500CS') THEN 'Postpaid Inflow M1 : TMH' --Mockup B2C+B2B
                    ELSE METRIC_NAME END METRIC_NAME
            , AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
            , SUM(ACTUAL_SNAP) ACTUAL_SNAP, SUM(TARGET_SNAP) TARGET_SNAP, MAX(PPN_TM) PPN_TM, MAX(LOAD_DATE) LOAD_DATE
            
        FROM AUTOKPI.ADHOC_RAW_SALE_PERFORMANCE_MTH
        WHERE METRIC_CD <> 'TB3R000602CS' --TOL Inflow M1 Connected : DataService (No Target)
        AND AREA_CD NOT IN ('016', '040')
            
        GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD--, METRIC_CD, METRIC_NAME
            , CASE 	WHEN METRIC_CD IN ('DB2R010500CS', 'DB2R020500CS') THEN 'DB2R000500CS' --Postpaid Inflow M1 : DTAC (Mockup B2C+B2B)
                    WHEN METRIC_CD IN ('TB2R010500CS', 'TB2R020500CS') THEN 'TB2R000500CS' --Postpaid Inflow M1 : TMH (Mockup B2C+B2B)
                    ELSE METRIC_CD END
            , CASE 	WHEN METRIC_CD IN ('DB2R010500CS', 'DB2R020500CS') THEN 'Postpaid Inflow M1 : DTAC' --Mockup B2C+B2B
                    WHEN METRIC_CD IN ('TB2R010500CS', 'TB2R020500CS') THEN 'Postpaid Inflow M1 : TMH' --Mockup B2C+B2B
                    ELSE METRIC_NAME END
            , AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
    ) -->> W_SALE_ACTUAL
    -----------------------------------------------------------------------------------------------------------------------

    , W_PRODUCT_AGG AS 
    (
        SELECT A.TM_KEY_YR, A.TM_KEY_QTR, A.TM_KEY_MTH, A.PRODUCT_GRP, A.COMP_CD, A.METRIC_CD, A.METRIC_NAME
            , CASE WHEN REGEXP_LIKE(A.METRIC_CD, 'B1|B2') THEN REGEXP_REPLACE(A.METRIC_CD, 'DB|TB', 'B') ELSE A.METRIC_CD END TMP_METRIC_CD
            , CASE WHEN REGEXP_LIKE(A.METRIC_CD, 'B1|B2') THEN REGEXP_REPLACE(A.METRIC_NAME, ' : DTAC| : TMH') ELSE A.METRIC_NAME END TMP_METRIC_NAME
            , A.AREA_NO, A.AREA_TYPE, A.AREA_CD, A.AREA_NAME
            , COALESCE(A.ACTUAL_SNAP, 0) AS ACTUAL_SNAP
            , COALESCE(A.TARGET_SNAP, 0) AS TARGET_SNAP
            , A.PPN_TM, A.LOAD_DATE
        
        FROM W_SALE_ACTUAL A
    ) -->> W_PRODUCT_AGG
    -----------------------------------------------------------------------------------------------------------------------

    , W_TOTAL_AGG AS 
    (	
        -->> ALL = (TRUE + DTAC)
        SELECT A.*
            , T.ACTUAL_SNAP AS T_ACTUAL, T.TARGET_SNAP AS T_TARGET
            , D.ACTUAL_SNAP AS D_ACTUAL, D.TARGET_SNAP AS D_TARGET
        
        FROM (
            SELECT TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, SUBSTR(TM_KEY_MTH, 5, 2) AS MTH
                , PRODUCT_GRP, 'ALL' AS COMP_CD, TMP_METRIC_CD AS METRIC_CD, TMP_METRIC_NAME AS METRIC_NAME
                , AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
                , SUM(ACTUAL_SNAP) AS ACTUAL, SUM(TARGET_SNAP) AS TARGET, MAX(PPN_TM) AS PPN_TM, MAX(LOAD_DATE) AS LOAD_DATE
            FROM W_PRODUCT_AGG
            GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, TMP_METRIC_CD, TMP_METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
        ) A
        
        LEFT JOIN ( -->> TRUE
            SELECT TM_KEY_MTH, TMP_METRIC_CD, AREA_CD, ACTUAL_SNAP, TARGET_SNAP
            FROM W_PRODUCT_AGG
            WHERE COMP_CD = 'TRUE'
        ) T
            ON A.TM_KEY_MTH = T.TM_KEY_MTH
            AND A.METRIC_CD = T.TMP_METRIC_CD
            AND A.AREA_CD = T.AREA_CD
            
        LEFT JOIN ( -->> DTAC
            SELECT TM_KEY_MTH, TMP_METRIC_CD, AREA_CD, ACTUAL_SNAP, TARGET_SNAP
            FROM W_PRODUCT_AGG
            WHERE COMP_CD = 'DTAC'
        ) D
            ON A.TM_KEY_MTH = D.TM_KEY_MTH
            AND A.METRIC_CD = D.TMP_METRIC_CD
            AND A.AREA_CD = D.AREA_CD
    ) -->> W_TOTAL_AGG
    -----------------------------------------------------------------------------------------------------------------------
    -----------------------------------------------------------------------------------------------------------------------

    -->> Output

    SELECT TM_KEY_YR AS YR, MTH, PRODUCT_GRP, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
        , ACHIEVEMENT, MOM, YOY
        -->> Sales Performance Color
        , CASE 	WHEN ACTUAL > TARGET THEN 'green'
                WHEN ACTUAL < TARGET AND ACTUAL > LAST_YR_ACTUAL THEN 'yellow'
                WHEN ACTUAL < TARGET AND ACTUAL < LAST_YR_ACTUAL THEN 'red'
                END SALE_PERF_COLOR
        , LAST_YR_ACTUAL, LAST_MTH_ACTUAL, ACTUAL, TARGET
        , T_ACTUAL, T_TARGET, D_ACTUAL, D_TARGET, LOAD_DATE
                
    FROM (
        SELECT TM_KEY_YR, TM_KEY_MTH, MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
            , LAST_YR_ACTUAL, LAST_MTH_ACTUAL, ACTUAL, TARGET
            -->> % Achievement
            , CASE 	WHEN COALESCE(ACTUAL,0) = 0 OR COALESCE(TARGET,0) = 0 THEN NULL 
                    ELSE ACTUAL / TARGET * 100 END ACHIEVEMENT
            -->> % MoM
            , CASE 	WHEN COALESCE(ACTUAL,0) = 0 OR COALESCE(LAST_MTH_ACTUAL,0) = 0 THEN NULL 
                    ELSE (ACTUAL - LAST_MTH_ACTUAL) / LAST_MTH_ACTUAL * 100 END MOM
            -->> % YoY
            , CASE 	WHEN COALESCE(ACTUAL,0) = 0 OR COALESCE(LAST_YR_ACTUAL,0) = 0 THEN NULL 
                    ELSE (ACTUAL - LAST_YR_ACTUAL) / LAST_YR_ACTUAL * 100 END YOY
            , T_ACTUAL, T_TARGET, D_ACTUAL, D_TARGET, LOAD_DATE
            
        FROM (
            SELECT TM_KEY_YR, TM_KEY_MTH, MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
                , LAG(ACTUAL) OVER(PARTITION BY METRIC_CD, AREA_CD, MTH ORDER BY TM_KEY_YR) LAST_YR_ACTUAL
                , LAG(ACTUAL) OVER(PARTITION BY METRIC_CD, AREA_CD ORDER BY TM_KEY_MTH) LAST_MTH_ACTUAL
                , ACTUAL, TARGET, T_ACTUAL, T_TARGET, D_ACTUAL, D_TARGET, LOAD_DATE
            FROM W_TOTAL_AGG
        ) AGG_1
        
    ) AGG_2

    WHERE TM_KEY_MTH BETWEEN 202504 AND 202505
    ORDER BY TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, AREA_NO, AREA_CD 
"""


# AKPIPRD
dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
conn = oracledb.connect(dsn)
print(f'\n{AKPIPRD_db} : Connected')
cur = conn.cursor()
op_dir = 'output'
op_file = 'sales_performance.xlsx'


try:
    print(f'\n Processing...')
    cur.execute(query)
    df = pd.DataFrame(cur.fetchall(), columns=['YR', 'MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACHIEVEMENT', 'MOM', 'YOY', 'SALE_PERF_COLOR', 'LAST_YR_ACTUAL', 'LAST_MTH_ACTUAL', 'ACTUAL', 'TARGET', 'T_ACTUAL', 'T_TARGET', 'D_ACTUAL', 'D_TARGET', 'LOAD_DATE'])
    # print(f'\n{df}')
    df.to_excel(f'{op_dir}/{op_file}', sheet_name='Data', index=False)
    print(f'\n -> Generate "{op_file}" successfully')


except oracledb.DatabaseError as e:
    print(f'Error with Oracle : {e}')


finally:
    cur.close()
    conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    print(f'\nJob Done !!!')


Job Start...

AKPIPRD : Connected

 Processing...

 -> Generate "sales_performance.xlsx" successfully

AKPIPRD : Disconnected

Job Done !!!


#### Test : Generate Rawdata (CSV)

In [4]:
# # Source File

# src_file = 'C:\Ruz\MyProject\Code\Jupyter\ETL\Adhoc\\rawdata\sales_performance_actual.csv'
# src_df = pd.read_csv(src_file)
# src_df = src_df.replace(np.nan, None)
# # src_df.head()
# # rows = [tuple(x) for x in src_df.values]